# Backticks Code Replacement Sandbox

The purpose of this notebook is to explore the functionality described in [this issue](https://github.com/open-resources/problem_bank_scripts/issues/35), started in [this PR](https://github.com/open-resources/problem_bank_scripts/pull/36) and continued [in this one](https://github.com/open-resources/problem_bank_scripts/pull/42).

For example, this
```
data2["params"]["part1"]["ans1"]["value"] = "Hello `x` world"
```

should get replaced with:
```
data2["params"]["part1"]["ans1"]["value"] = "Hello <code>x</code> world"
```

Having something we can conchtmlely test will also help us debug future cases where the current functionality breaks and easily adapt/change the behaviour of this function as the PrairieLearn markdown rendering functionality improves.


## Function that does the conversion

In [1]:
import re


def backticks_to_code_tags(html: str) -> str:
    """
    Converts backticks to <code> tags, and code fences to <pl-code> tags.

    Args:
        html (str): The HTML to convert.

    """
    # Because these regexps could be run an arbitrary number of times, its slightly more efficient to compile them once,
    #  since  they will always be compiled internally by re if we pass them as strings.
    html = re.sub(
        r"```(?P<language>\w+)?(?(language)(\{(?P<highlighting>[\d,-]*)\})?|)(?P<Code>[^`]+)```",
        r'<pl-code language="\g<language>" highlight-lines="\g<highlighting>">\g<Code></pl-code>',
        html,
        flags=re.MULTILINE,
    )
    html = html.replace(' language=""', "")  # Remove empty language attributes
    html = html.replace(
        ' highlight-lines=""', ""
    )  # Remove empty highlight-lines attributes
    html = re.sub(r"`(?P<Code>[^`]+)`", r"<code>\g<Code></code>", html)
    return html

## Case 1: single case of backticks:

Test cases:

- "Hello `x+1` world"
- "`x+1`"
- `x+1`

In [2]:
# Test backticks_to_code_tags here
for item in ['"Hello `x+1` world"', '"`x+1`"', "`x+1`"]:
    print(backticks_to_code_tags(item))

"Hello <code>x+1</code> world"
"<code>x+1</code>"
<code>x+1</code>


## Case 2: multiple instances of backticks

Test cases:


- "Hello `x+1` and `y+5` world"
- `x+1` `y+5`


In [3]:
# Test backticks_to_code_tags here
for item in ['"Hello `x+1` and `y+5` world"', '"`x+1` `y+5`"']:
    print(backticks_to_code_tags(item))

"Hello <code>x+1</code> and <code>y+5</code> world"
"<code>x+1</code> <code>y+5</code>"


## Case 3: Codefence

Test cases:

- see multi-line string below

In [4]:
print(
    backticks_to_code_tags(
        """
```
for i in range(5):

    print(i)
    print(f'hello work {i}')

```"""
    )
)


<pl-code>
for i in range(5):

    print(i)
    print(f'hello work {i}')

</pl-code>


In [5]:
print(
    backticks_to_code_tags(
        """ Code block option:
```python
for i in range(5):

    print(i)
    print(f'hello work {i}')

```
"""
    )
)

 Code block option:
<pl-code language="python">
for i in range(5):

    print(i)
    print(f'hello work {i}')

</pl-code>



In [6]:
print(
    backticks_to_code_tags(
        """ Code block option:
```python{1,3-4}
for i in range(5):

    print(i)
    print(f'hello work {i}')

```
Code block option end
"""
    )
)

 Code block option:
<pl-code language="python" highlight-lines="1,3-4">
for i in range(5):

    print(i)
    print(f'hello work {i}')

</pl-code>
Code block option end

